<a href="https://colab.research.google.com/github/hifistelija/ML_Predictions/blob/main/ML_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kirjastojen tuonti

In [ ]:
import pandas as pd
import numpy as np

# Datan sisäänluku

In [ ]:
df = pd.read_excel("ML_Määrät.xlsx")

In [ ]:
df.head()

,Kirjauspvm.,Linjakoodi,Päivän nimi,Varaukset,Nimikkeen nro,Kuvaus,Määrä,Perusmittayksikkö
0,2023-07-12,274,Keskiviikko,263.0,R02038,Juustoreissari itsetehty,3.0,KPL
1,2023-07-12,274,Keskiviikko,263.0,R00481,Kahvi iso,14.0,KPL
2,2023-07-12,274,Keskiviikko,263.0,R00339,Vesi kuuma,2.0,KPL
3,2023-07-12,274,Keskiviikko,263.0,R00025,Kahvi,43.0,KPL
4,2023-07-12,274,Keskiviikko,263.0,R00001,Santsikahvi,4.0,KPL


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458505 entries, 0 to 458504
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Kirjauspvm.        458505 non-null  datetime64[ns]
 1   Linjakoodi         458505 non-null  int64         
 2   Päivän nimi        458505 non-null  object        
 3   Varaukset          455376 non-null  float64       
 4   Nimikkeen nro      458505 non-null  object        
 5   Kuvaus             458505 non-null  object        
 6   Määrä              458492 non-null  float64       
 7   Perusmittayksikkö  458505 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.0+ MB


# Datan siivous ja poikkeamien korjaus

In [ ]:
df = df.dropna()

In [ ]:
def replace_with_robust_zscore(df):
  median = df["Määrä"].median()
  mad = np.median(np.abs(df["Määrä"] - median))
  z_score = 0.6745 * (df["Määrä"] - median) / mad
  df["Määrä"] = np.where(np.abs(z_score) > 3, median, df["Määrä"])
  return df


df = df.groupby(
    ["Nimikkeen nro", "Päivän nimi", "Linjakoodi"]).apply(replace_with_robust_zscore)

<ipython-input-6-585034dd096e>:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  ["Nimikkeen nro", "Päivän nimi", "Linjakoodi"]).apply(replace_with_robust_zscore)


# Teksti sarakkeiden koodaus

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le_paivan_nimi = LabelEncoder()
le_nimikkeen_nro = LabelEncoder()

In [ ]:
df["Päivän nimi"] = le_paivan_nimi.fit_transform(df["Päivän nimi"])
df["Nimikkeen nro"] = le_nimikkeen_nro.fit_transform(df["Nimikkeen nro"])

In [ ]:
X = df.drop(["Määrä", "Perusmittayksikkö",  "Kuvaus", "Kirjauspvm."], axis=1 )
y = df["Määrä"]

# Mallin luonti

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_model = RandomForestRegressor(
    n_estimators=1000,
    min_samples_split=15,
    min_samples_leaf=1,
    max_features=1.0,
    max_depth=None,
    bootstrap=True
)

In [ ]:
rf_model.fit(X,y)

RandomForestRegressor(min_samples_split=15, n_estimators=1000, random_state=42)

# Uuden datan tuonti ennustetta varten

In [ ]:
new_data = pd.read_excel("ML_uusi_data.xlsx")

In [ ]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50570 entries, 0 to 50569
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Linjakoodi         50570 non-null  int64  
 1   Päivän nimi        50570 non-null  object 
 2   Ennuste            49675 non-null  float64
 3   Nimikkeen nro      50570 non-null  object 
 4   Kuvaus             50570 non-null  object 
 5   Perusmittayksikkö  50570 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.3+ MB


In [ ]:
new_data_copy = new_data # kopio dataframesta poistettujen sarakkeiden uudelleen tuontia varten

# Uuden datan muokkaukset

In [ ]:
new_data = new_data.dropna()

In [ ]:
new_data.rename(columns={"Ennuste": "Varaukset"}, inplace=True)

<ipython-input-18-00b4c3c0014e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.rename(columns={"Ennuste": "Varaukset"}, inplace=True)


In [ ]:
new_data["Päivän nimi"] = le_paivan_nimi.transform(new_data["Päivän nimi"])
new_data["Nimikkeen nro"] = le_nimikkeen_nro.transform(new_data["Nimikkeen nro"])

<ipython-input-19-bd5520728f0c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["Päivän nimi"] = le_paivan_nimi.transform(new_data["Päivän nimi"])
<ipython-input-19-bd5520728f0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["Nimikkeen nro"] = le_nimikkeen_nro.transform(new_data["Nimikkeen nro"])


In [ ]:
columns_to_drop = ["Perusmittayksikkö", "Kuvaus"]  # list of columns to drop

In [ ]:
for col in columns_to_drop:
  if col in new_data.columns:
    new_data.drop(col, axis=1, inplace=True)

<ipython-input-21-46c3b6d70bb9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.drop(col, axis=1, inplace=True)
<ipython-input-21-46c3b6d70bb9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.drop(col, axis=1, inplace=True)


In [ ]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49675 entries, 0 to 50563
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Linjakoodi     49675 non-null  int64  
 1   Päivän nimi    49675 non-null  int64  
 2   Varaukset      49675 non-null  float64
 3   Nimikkeen nro  49675 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 1.9 MB


# Menekki ennuste

In [ ]:
predictions = rf_model.predict(new_data)

In [ ]:
new_data["Määrä"] = predictions

<ipython-input-24-48eaa704a70a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["Määrä"] = predictions


In [ ]:
new_data.head(10)

,Linjakoodi,Päivän nimi,Varaukset,Nimikkeen nro,Määrä
0,47,4,492.0,84,3.842447
1,47,4,492.0,66,2.945044
2,47,4,492.0,179,5.745889
3,47,4,492.0,155,23.668421
4,47,4,492.0,99,12.696190
5,47,4,492.0,91,4.525413
6,47,4,492.0,85,3.172574
7,47,4,492.0,81,11.834635
8,47,4,492.0,65,1.696795
9,47,4,492.0,223,2.949431


In [ ]:
new_data["Päivän nimi"] = le_paivan_nimi.inverse_transform(new_data["Päivän nimi"])
new_data["Nimikkeen nro"] = le_nimikkeen_nro.inverse_transform(new_data["Nimikkeen nro"])

<ipython-input-26-68818545289d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["Päivän nimi"] = le_paivan_nimi.inverse_transform(new_data["Päivän nimi"])
<ipython-input-26-68818545289d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["Nimikkeen nro"] = le_nimikkeen_nro.inverse_transform(new_data["Nimikkeen nro"])


In [ ]:
new_data.head(10)

,Linjakoodi,Päivän nimi,Varaukset,Nimikkeen nro,Määrä
0,47,Sunnuntai,492.0,40327,3.842447
1,47,Sunnuntai,492.0,30309,2.945044
2,47,Sunnuntai,492.0,R01180,5.745889
3,47,Sunnuntai,492.0,R00025,23.668421
4,47,Sunnuntai,492.0,50130,12.696190
5,47,Sunnuntai,492.0,40423,4.525413
6,47,Sunnuntai,492.0,40331,3.172574
7,47,Sunnuntai,492.0,40169,11.834635
8,47,Sunnuntai,492.0,30308,1.696795
9,47,Sunnuntai,492.0,R01871,2.949431


In [ ]:
new_data.to_excel("ML_myynti_ennuste.xlsx", index=False)